In [ ]:
import pandas as pd

# Reading large line-by-line JSON file

In [ ]:
TWEETS_JSON_FILE = 'geotagged_tweets_20160812-0912.jsons'

In [ ]:
# Since file size of the tweets is large, pandas needs to read the file in chunks.
chunked_reader = pd.read_json(TWEETS_JSON_FILE, 
                                lines=True, 
                                orient='records', 
                                chunksize=50000)
chunked_reader

In [ ]:
# Initialize an empty dataframe on which the chunks can be appended
df_tweets_raw = pd.DataFrame()

# Append chunks to dataframe
i = 1
for chunk in chunked_reader:
    df_tweets_raw = df_tweets_raw.append(chunk)
    print(i,'chunk read')
    i += 1

# Unnest columns
As the original JSON has multi-level nesting, and pandas only reads the first level, we need to do the rest ourselves...

In [ ]:
def unnest_column(df, prefix):
    '''Creates separate columns in the dataframe from a single column containing a dict. 
        Returns a dataframe.
    '''
    
    # get the fields from inside the nested column (== keys of dict)
    keys = list(df.iloc[0][prefix].keys())
    
    # Initialize the new unnested columns
    for key in keys:
        df[ prefix + '_' + key ] = np.nan
    print('initialized empty columns for', prefix)
        
    # Fill the new columns by apply'ing
    for key in keys:
        print('unnesting', key,'from',prefix)
        df[prefix+'_'+key] = df[prefix].apply(lambda x: x[key] if type(x) == type(dict()) and key in x else np.nan)
    
    return df

In [ ]:
# Unnest the place columns into place_*
df_tweets_raw = unnest_column(df_tweets_raw, 'place')
df_tweets_raw.columns